In [5]:
# 비트코인 API
import requests as req
url = "https://api.bithumb.com/v1/candles/minutes/1?market=KRW-BTC&count=1"
d_url = "https://api.bithumb.com/v1/candles/days?market=KRW-BTC&count=1"
ori = req.get(url).json()[0]
ori2 = req.get(d_url).json()[0]

# '''분봉'''
# 시작가
openp = float(ori['opening_price'])
# 최고가
maxp = float(ori['high_price'])
# 최소가
minp = float(ori['low_price'])
# 변화폭
bandp = maxp - minp

# '''일봉'''
# 시작가
openp2 = float(ori2['opening_price'])
# 최고가
maxp2 = float(ori2['high_price'])
# 최소가
minp2 = float(ori2['low_price'])
# 변화폭
bandp2 = maxp2 - minp2


print("분기준 시작가:",openp,"\n최대가:",maxp,"\n최소가:",minp,"\n변화폭:",bandp)
print("일기준 시작가:",openp2,"\n최대가:",maxp2,"\n최소가:",minp2,"\n변화폭:",bandp2)
# print("검토:", openp + bandp)
# print("최고가:", maxp)

# 파이썬 외 언어, 삼항연산자 ((maxp - (openp + bandp)) > 0) ? "떡상중" : "떡락중")
result = "떡상중" if ((maxp - (openp + bandp)) > 0) else "떡락중"
result2 = "떡상중" if ((maxp2 - (openp2 + bandp2)) > 0) else "떡락중"
print("분봉으로 판단:", result)
print("일봉으로 판단:", result2)

분기준 시작가: 97588000.0 
최대가: 97720000.0 
최소가: 97408000.0 
변화폭: 312000.0
일기준 시작가: 101900000.0 
최대가: 102129000.0 
최소가: 88999000.0 
변화폭: 13130000.0
분봉으로 판단: 떡락중
일봉으로 판단: 떡락중


In [ ]:
# 비트코인 원화
import requests
import time

# ──────────────── 설정 ────────────────
USE_UPBIT_AS_PRIMARY = True       # True 추천 (한국 실시간 KRW 가격)
FALLBACK_LAST_RATE = 1450.0       # 매우 오래된 경우 쓸 임의 환율 (2026년 초 평균 근처)

last_usd_krw = None               # 글로벌 캐시

def get_usd_krw():
    """실패 시 None 반환"""
    try:
        # exchangerate.host (현재 제한적임 → 대안으로 바꾸는 게 좋음)
        url = "https://api.exchangerate.host/latest?base=USD&symbols=KRW"
        resp = requests.get(url, timeout=5)
        data = resp.json()
        if 'rates' in data and 'KRW' in data['rates']:
            return float(data['rates']['KRW'])
    except Exception as e:
        print(f"환율 API 오류: {e}")
    return None

def safe_get_usd_krw():
    """안전하게 환율 가져오기 + 캐싱"""
    global last_usd_krw
    
    rate = get_usd_krw()
    if rate is not None and rate > 500:   # 비현실적인 값 방지 (예: 0이나 999999)
        last_usd_krw = rate
        return rate
    
    # 실패 시 마지막 값 사용 (있으면)
    if last_usd_krw is not None:
        print("환율 API 실패 → 마지막 저장값 사용")
        return last_usd_krw
    
    # 처음부터 실패하면 fallback
    print("환율 정보 전혀 없음 → 임시값 사용")
    return FALLBACK_LAST_RATE


# ──────────────── Upbit 버전 (가장 추천) ────────────────
def get_btc_krw_upbit():
    try:
        url = "https://api.upbit.com/v1/ticker?markets=KRW-BTC"
        resp = requests.get(url, timeout=5)
        data = resp.json()
        if resp.status_code == 200 and data:
            return float(data[0]['trade_price'])
    except Exception as e:
        print(f"Upbit 오류: {e}")
    return None


# ──────────────── Binance + 환율 버전 (필요 시) ────────────────
def get_btc_krw_binance():
    try:
        btc_usd = float(requests.get(
            "https://api.binance.com/api/v3/ticker/price?symbol=BTCUSDT",
            timeout=5
        ).json()['price'])
        
        rate = safe_get_usd_krw()
        return btc_usd * rate
    except Exception as e:
        print(f"Binance 오류: {e}")
        return None


# 테스트용 예시
if __name__ == "__main__":
    for _ in range(100):
        if USE_UPBIT_AS_PRIMARY:
            price = get_btc_krw_upbit()
            source = "Upbit"
        else:
            price = get_btc_krw_binance()
            source = "Binance + cached rate"
        
        if price:
            print(f"[{source}] BTC/KRW : {int(price):,} 원")
        else:
            print("가격 조회 완전 실패")
        
        time.sleep(10)

[Upbit] BTC/KRW : 97,053,000 원
[Upbit] BTC/KRW : 97,177,000 원
[Upbit] BTC/KRW : 97,221,000 원
[Upbit] BTC/KRW : 97,264,000 원
[Upbit] BTC/KRW : 97,196,000 원
[Upbit] BTC/KRW : 97,086,000 원
[Upbit] BTC/KRW : 97,079,000 원
[Upbit] BTC/KRW : 97,031,000 원
[Upbit] BTC/KRW : 97,040,000 원
[Upbit] BTC/KRW : 97,020,000 원
[Upbit] BTC/KRW : 96,996,000 원
[Upbit] BTC/KRW : 96,991,000 원
[Upbit] BTC/KRW : 97,131,000 원
[Upbit] BTC/KRW : 97,173,000 원
[Upbit] BTC/KRW : 97,336,000 원
[Upbit] BTC/KRW : 97,489,000 원


In [1]:
# 비트코인 미국달러 환산
import requests
import time

# ──────────────── 설정 ────────────────
def get_btc_usd():
    """Binance에서 BTC/USD 실시간 가격 가져오기"""
    try:
        url = "https://api.binance.com/api/v3/ticker/price?symbol=BTCUSDT"
        resp = requests.get(url, timeout=5)
        data = resp.json()
        return float(data['price'])
    except Exception as e:
        print(f"Binance API 오류: {e}")
        return None

# 테스트용 예시 (주피터나 .py에서 실행)
if __name__ == "__main__":
    print("BTC/USD 가격 감시 시작 (Binance 기준)")
    print("가격은 약 10초마다 업데이트됩니다.\n")
    
    for i in range(100):
        price = get_btc_usd()
        
        if price is not None:
            # 소수점 2자리까지 표시 (실제 거래소처럼)
            formatted = f"${price:,.2f}"
            print(f"[{i+1:02d}] BTC/USD : {formatted}")
        else:
            print(f"[{i+1:02d}] 가격 조회 실패... 재시도 중")
        
        time.sleep(10)  # 10초 간격 (API 부하 줄이기)

BTC/USD 가격 감시 시작 (Binance 기준)
가격은 약 10초마다 업데이트됩니다.

[01] BTC/USD : $65,101.72
[02] BTC/USD : $65,106.00
[03] BTC/USD : $65,084.93
[04] BTC/USD : $65,030.36
[05] BTC/USD : $65,004.63
[06] BTC/USD : $64,989.06
[07] BTC/USD : $65,017.45
[08] BTC/USD : $65,051.61
[09] BTC/USD : $65,002.84
[10] BTC/USD : $64,999.20
[11] BTC/USD : $64,973.65
[12] BTC/USD : $64,923.42
[13] BTC/USD : $64,889.12
[14] BTC/USD : $64,800.01
[15] BTC/USD : $64,795.73
[16] BTC/USD : $64,763.03
[17] BTC/USD : $64,731.69
[18] BTC/USD : $64,781.94
[19] BTC/USD : $64,739.24
[20] BTC/USD : $64,766.37
[21] BTC/USD : $64,795.28
[22] BTC/USD : $64,785.84
[23] BTC/USD : $64,835.14
[24] BTC/USD : $64,794.81
[25] BTC/USD : $64,851.97
[26] BTC/USD : $64,874.50
[27] BTC/USD : $64,864.72
[28] BTC/USD : $64,811.33
[29] BTC/USD : $64,765.59
[30] BTC/USD : $64,790.60
[31] BTC/USD : $64,720.40
[32] BTC/USD : $64,722.49
[33] BTC/USD : $64,711.01
[34] BTC/USD : $64,714.49
[35] BTC/USD : $64,834.82
[36] BTC/USD : $64,838.61
[37] BTC/USD

In [4]:
# 판단하기

# 개인스킬
Me = 'JAVA'

# 모집공고 요구스킬
CompA = ['JAVA', 'PYTHON','HTML','Javascript','C','ORACLE']
CompB = ['HTML', 'PyThon', 'C', 'MySQL']
CompC = ['java','html','CSS','Javascript','C++','ORACLE']

print("합격" if Me in CompA else "불합격")
print("합격" if Me in CompB else "불합격")
print("합격" if Me in CompC else "불합격")

합격
불합격
불합격


In [9]:
# 정제하기(국제우주정거장 인공위성 실시간 위치)
import requests as req
import webbrowser as wb

# 위성위치
url = "http://api.open-notify.org/iss-now.json"
# 탑승자 명단
url2 = "http://api.open-notify.org/astros.json"

result = req.get(url).json()
lat = result['iss_position']['latitude'] # 위도
lon = result['iss_position']['longitude'] # 경도

human_num = req.get(url2).json()["number"]

url = f"https://www.google.com/maps?q={lat},{lon}"
print(f"현재 {human_num}명이 탑승한 인공위성(국제우주정거장) 위치를 열겠습니다.위도:{lat}, 경도:{lon}")
wb.open(url)

현재 12명이 탑승한 인공위성(국제우주정거장) 위치를 열겠습니다.위도:4.8552, 경도:42.3316


True

gio: https://www.google.com/maps?q=4.8552,42.3316: Operation not supported


In [1]:
# 문제1: 지역 날씨 (부경대 용당캠퍼스 용당 4관 - 대략 위도 35.11611, 경도 129.09167)
import urllib.request
import json

weather_url = "https://api.open-meteo.com/v1/forecast?latitude=35.11611&longitude=129.09167&current_weather=true"

with urllib.request.urlopen(weather_url) as response:
    weather_data = json.loads(response.read().decode())
    
# 딕셔너리 분석 및 의미있는 결과 출력
current_weather = weather_data.get('current_weather', {})
print("문제1: 부경대 용당캠퍼스 현재 날씨")
print(f"온도: {current_weather.get('temperature')} °C")
print(f"풍속: {current_weather.get('windspeed')} km/h")
print(f"풍향: {current_weather.get('winddirection')} °")
print(f"날씨 코드: {current_weather.get('weathercode')} (0: 맑음, 1-3: 구름, 45: 안개, 51-55: 이슬비, 등)")
print("---")

문제1: 부경대 용당캠퍼스 현재 날씨
온도: -0.2 °C
풍속: 14.3 km/h
풍향: 313 °
날씨 코드: 0 (0: 맑음, 1-3: 구름, 45: 안개, 51-55: 이슬비, 등)
---


In [6]:
# 문제2: FDA 데이터베이스
fda_url = "https://api.fda.gov/drug/event.json?count=patient.reaction.reactionmeddrapt.exact"

with urllib.request.urlopen(fda_url) as response:
    fda_data = json.loads(response.read().decode())

meta = fda_data.get('meta', {})
results = fda_data.get('results', [])

# 결과를 딕셔너리로 매핑 (term: count)
reaction_counts = {item['term']: item['count'] for item in results}

nausea_count = reaction_counts.get('NAUSEA', 0)
print("NAUSEA 보고 건수:", nausea_count)

covid_count = reaction_counts.get('COVID-19', 0)
print("COVID-19 보고 건수:", covid_count)

mi_count = reaction_counts.get('MYOCARDIAL INFARCTION', 0)
print("MYOCARDIAL INFARCTION 보고 건수:", mi_count)

print("COVID-19와 심근경색(MYOCARDIAL INFARCTION) 의 비교", "코로나가 더 많음" if covid_count > mi_count else "심근경색이 더 많음")

# NAUSEA와 VOMITING 비교
vomiting_count = reaction_counts.get('VOMITING', 0)
if nausea_count > vomiting_count:
    more = "NAUSEA가 더 많음"
    diff = nausea_count - vomiting_count
else:
    more = "VOMITING이 더 많음"
    diff = vomiting_count - nausea_count
print(f"문제2-5: {more}, 차이: {diff} 건")

# disclaimer에 "FDA" 포함 여부
disclaimer = meta.get('disclaimer', '')
has_fda = "FDA" in disclaimer
print("disclaimer에 'FDA' 포함:", has_fda)

NAUSEA 보고 건수: 752660
COVID-19 보고 건수: 171703
MYOCARDIAL INFARCTION 보고 건수: 171221
COVID-19와 심근경색(MYOCARDIAL INFARCTION) 의 비교 코로나가 더 많음
문제2-5: NAUSEA가 더 많음, 차이: 305998 건
disclaimer에 'FDA' 포함: True
